## Cleaning comapany review data
There are a 2 main issuese in the data that requires cleaning.
1. Similar industry labels. E.g. "financial services" and "financial services, financial services"/
2. part of role being found in location, and location being found in date and some date are missing due to the pattern used to extract these info
<br>
For issue 1, manually inspect the csv file using filters and assess the industry labels that should be grouped together, and overwrite them with the more suitable label.<br>
For issue 2. the data cleaning steps will be as shown below


In [20]:
import pandas as pd
df = pd.read_csv("company_review_dataset_to_clean.csv")
df.head()

,company,industry,role,location,date,review,main_rating,work_life_balance_rating,salary_and_benefits_rating,job_security_and_advancement_rating,management_rating,job_culture_rating
0,Walmart,Retail & Wholesale,Frozen / Dairy (Former Employee),"Singapore, S00","May 27, 2014",A typical day at work consist of stocking the...,4,2,3,5,3,4
1,Walmart,Retail & Wholesale,Proactive care (Current Employee),Home,"June 17, 2022",WFH proactive care in the company is amazing a...,5,5,5,5,5,5
2,Walmart,Retail & Wholesale,Self Checkout Host (Current Employee),761 Airport Beltway,"May 2, 2020",I am a self checkout host and I get paid $11.0...,5,5,5,0,5,5
3,Amazon,Retail & Wholesale,driver (Former Employee),sg,"March 22, 2023",typical day will go collect parcels and deliv...,4,3,3,3,3,3
4,Amazon,Retail & Wholesale,Customer Service Executive (Former Employee),Singapore,"May 24, 2023",Over all good experience. Friendly work enviro...,5,0,0,0,0,0


In [21]:
df.shape

(12116, 12)

Besides cleaning the data, we will also be extract the current status to create a new column called current status. Current status information are found in bracket form and are usually (Current Employee, Former Employee or Intern)

In [22]:
#Assessing the number of dirty roles. These are defined as role contents with more than 1 bracket, implying that
#the reviewer put further information in brackets in additional to the current status information being in brackets.
def assess_role(x):
    count=0
    for i in x:
        if i=='(':
            count+=1
    if count >=2:
        return 1
    else:
        return 0
df["dirty_roles"] = df["role"].apply(lambda x: assess_role(x))

In [23]:
df["dirty_roles"].value_counts()

0    11644
1      472
Name: dirty_roles, dtype: int64

In [47]:

#Clean and extract infortion for new current status column
def check_current_status(x):
    reference = ["Intern", "internship","Former Employee" , "Current Employee"]
    for i in reference:
        if i.lower() in x.lower():
            return 1    
    return 0

def extract_current_status(x):
    try:
        reference = ["Intern", "internship","Former Employee", "Current Employee"]
        for i in reference:
            if i.lower() in x.lower():
                return i.lower()
    except:
        return x
def replace_them(x):
    try:
        x=x.lower()
        reference = ["(Intern)","(Internship)","(Former Employee)", "(Current Employee)","Former Employee", "Intern", "Current Employee"]
        for i in reference:
            i2=i.lower()
            if i2 in x:
                x_new=x.replace( i2 , "")
                return x_new
        return x
    except:
        return x
    

    #1 means the key words for current status are found in role, if it isnt in role, i can only be in location
df["check_current_status"]=df["role"].apply(lambda x: check_current_status(x))
df["check_current_status"].value_counts()
df[df["check_current_status"]==0]

#to clean
#break the data into 2 different sets to clean.
#Implies key current status words are found in role, extracting the keyword is thus relatively straight forward
df1=df[df["check_current_status"]==1].reset_index(drop = True)

#this means the key words are found in location, where the cleaning steps are a little different.
#For the keywords that are found in date, these are cleaned in the csv manually
df2=df[df["check_current_status"]==0].reset_index(drop = True)

#to clean if the employment status is in location
#create a new column for current status called employment status, extract the terms and put inside there
#join role and location to be role with "-" as this was the separator pattern used during scrapping to extract info. 
df2["current_status"] = df2["location"].apply(lambda x: extract_current_status(x))
#as we have already extract current status into the current status column, we do not need them where they were initially found.
df2["location"] = df2["location"].apply(lambda x: replace_them(x))
#now connect role with location as role have been break into role and location columns
df2["role"] = df2["role"] + "-" + df2["location"]
df2["location"] = df2["date"]
df2["date"] = "Nil"
df2["role"] = df2["role"].apply(lambda x: replace_them(x))
df2.head()

#straight forward extraction of current status from role. This is the pattern for most of the data. 
df1["current_status"] = df1["role"].apply(lambda x: extract_current_status(x)) 
df1["role"] = df1["role"].apply(lambda x: replace_them(x))
df1.head()
### After cleaning, remeber to open the csv to make some changes as there are a variety of ways people put their
#role, current status, job location and date.

,company,industry,role,location,date,review,main_rating,work_life_balance_rating,salary_and_benefits_rating,job_security_and_advancement_rating,management_rating,job_culture_rating,dirty_roles,check_current_status,current_status
0,Walmart,Retail & Wholesale,frozen / dairy,"Singapore, S00","May 27, 2014",A typical day at work consist of stocking the...,4,2,3,5,3,4,0,1,former employee
1,Walmart,Retail & Wholesale,proactive care,Home,"June 17, 2022",WFH proactive care in the company is amazing a...,5,5,5,5,5,5,0,1,current employee
2,Walmart,Retail & Wholesale,self checkout host,761 Airport Beltway,"May 2, 2020",I am a self checkout host and I get paid $11.0...,5,5,5,0,5,5,0,1,current employee
3,Amazon,Retail & Wholesale,driver,sg,"March 22, 2023",typical day will go collect parcels and deliv...,4,3,3,3,3,3,0,1,former employee
4,Amazon,Retail & Wholesale,customer service executive,Singapore,"May 24, 2023",Over all good experience. Friendly work enviro...,5,0,0,0,0,0,0,1,former employee


In [50]:
df_final = pd.concat([df1,df2]).reset_index(drop = True)
df_final.drop(["dirty_roles", "check_current_status"], axis =1, inplace=True)
df_final.head(5)

,company,industry,role,location,date,review,main_rating,work_life_balance_rating,salary_and_benefits_rating,job_security_and_advancement_rating,management_rating,job_culture_rating,current_status
0,Walmart,Retail & Wholesale,frozen / dairy,"Singapore, S00","May 27, 2014",A typical day at work consist of stocking the...,4,2,3,5,3,4,former employee
1,Walmart,Retail & Wholesale,proactive care,Home,"June 17, 2022",WFH proactive care in the company is amazing a...,5,5,5,5,5,5,current employee
2,Walmart,Retail & Wholesale,self checkout host,761 Airport Beltway,"May 2, 2020",I am a self checkout host and I get paid $11.0...,5,5,5,0,5,5,current employee
3,Amazon,Retail & Wholesale,driver,sg,"March 22, 2023",typical day will go collect parcels and deliv...,4,3,3,3,3,3,former employee
4,Amazon,Retail & Wholesale,customer service executive,Singapore,"May 24, 2023",Over all good experience. Friendly work enviro...,5,0,0,0,0,0,former employee


In [51]:
df_final.to_csv("clean_indeed_data.csv", index = False)